# Final Project - Programming For Data Science

## Member Information
| Name              | ID       |
|-------------------|----------|
| Tran Nguyen Huan  | 21127050 |
| Nguyen Phat Dat   | 21127240 |


## Table of Contents:

1. [Acknowledgement](#acknowledgement)

2. [Introduction](#introduction)
    
    1. [About dataset](#about-dataset)


3. [Details](#details)

    1. [Collecting data](#1-collecting-data)

    2. [Exploring and Preprocessing data](#2-exploring-and-preprocessing-data)

    3. [Asking meaningful question](#3-asking-meaningful-questions)

    4. [Reflection](#reflection)

    5. [References](#references)

## Acknowledgement

# Introduction

## About dataset

## Details

### 1. Collecting data

### 2. Exploring and Preprocessing data

### 3. Asking meaningful questions

#### Question 1:

##### Preprocess

##### Analyze:

#### Question 2:

##### Preprocess

##### Analyze:

#### Question 3:

##### Preprocess

##### Analyze

#### Question 4:

##### Preprocess

##### Analyze

## Reflection

## References